# Transformers

## Importing necessary libraries

In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras import layers, models
import math

## Defining the inputs and variables

In [2]:
n_seq = 7
x = tf.constant(np.random.normal(size=(1,n_seq,512)), dtype='float32')
Wq = tf.Variable(np.random.normal(size=(512,512)), dtype='float32')
Wk = tf.Variable(np.random.normal(size=(512,512)), dtype='float32')
Wv = tf.Variable(np.random.normal(size=(512,512)), dtype='float32')

print('x.shape={}'.format(x.shape))
print('Wq.shape={}'.format(Wq.shape))
print('Wk.shape={}'.format(Wk.shape))
print('Wv.shape={}'.format(Wv.shape))

x.shape=(1, 7, 512)
Wq.shape=(512, 512)
Wk.shape=(512, 512)
Wv.shape=(512, 512)


## Defining the SelfAttention layer

In [46]:
class SelfAttentionLayer(layers.Layer):
    
    def __init__(self, d):
        super(SelfAttentionLayer, self).__init__()
        self.d = d
    
    def build(self, input_shape):
        self.Wq = self.add_weight(
            shape=(input_shape[-1], self.d), initializer='glorot_uniform',
            trainable=True, dtype='float32'
        )        
        self.Wk = self.add_weight(
            shape=(input_shape[-1], self.d), initializer='glorot_uniform',
            trainable=True, dtype='float32'
        )
        self.Wv = self.add_weight(
            shape=(input_shape[-1], self.d), initializer='glorot_uniform',
            trainable=True, dtype='float32'
        )
    
    def call(self, q_x, k_x, v_x):
        q = tf.matmul(q_x,self.Wq)
        k = tf.matmul(k_x,self.Wk)
        v = tf.matmul(v_x,self.Wv)

        p = tf.nn.softmax(tf.matmul(q, k, transpose_b=True)/math.sqrt(self.d))
        p = tf.squeeze(p)

        h = tf.matmul(p, v)
        return h,p

layer = SelfAttentionLayer(512)
h, p = layer(x, x, x)
print(h.shape)

(1, 7, 512)


## Using the Self-Attention in a Multi-head Attention Layer

In [47]:
multi_attn_head = [SelfAttentionLayer(64) for i in range(8)]
outputs = [head(x, x, x)[0] for head in multi_attn_head]

outputs = tf.concat(outputs, axis=-1)
print(outputs.shape)

(1, 7, 512)


## Defining the Fully-Connected Layer

In [48]:
class FCLayer(layers.Layer):
    
    def __init__(self, d1, d2):
        super(FCLayer, self).__init__()
        self.d1 = d1
        self.d2 = d2
    
    def build(self, input_shape):
        self.W1 = self.add_weight(
            shape=(input_shape[-1], self.d1), initializer='glorot_uniform',
            trainable=True, dtype='float32'
        )
        self.b1 = self.add_weight(
            shape=(self.d1,), initializer='glorot_uniform',
            trainable=True, dtype='float32'
        )        
        self.W2 = self.add_weight(
            shape=(input_shape[-1], self.d2), initializer='glorot_uniform',
            trainable=True, dtype='float32'
        )
        self.b2 = self.add_weight(
            shape=(self.d2,), initializer='glorot_uniform',
            trainable=True, dtype='float32'
        )  
    
    def call(self, x):
        ff1 = tf.nn.relu(tf.matmul(x,self.W1)+self.b1)
        ff2 = tf.matmul(x,self.W2)+self.b2
        return ff2
    
ff = FCLayer(2048, 512)(h)
print(ff.shape)

(1, 7, 512)


## Defining a SelfAttention layer with Masking Capability

In [49]:
"i kicked the ball and it disappeared"
class SelfAttentionLayer(layers.Layer):
    
    def __init__(self, d):
        super(SelfAttentionLayer, self).__init__()
        self.d = d
    
    def build(self, input_shape):
        self.Wq = self.add_weight(
            shape=(input_shape[-1], self.d), initializer='glorot_uniform',
            trainable=True, dtype='float32'
        )        
        self.Wk = self.add_weight(
            shape=(input_shape[-1], self.d), initializer='glorot_uniform',
            trainable=True, dtype='float32'
        )
        self.Wv = self.add_weight(
            shape=(input_shape[-1], self.d), initializer='glorot_uniform',
            trainable=True, dtype='float32'
        )
    
    def call(self, q_x, k_x, v_x, mask=None):
        q = tf.matmul(q_x,self.Wq) #[None, t, d]
        k = tf.matmul(k_x,self.Wk) #[None, t, d]
        v = tf.matmul(v_x,self.Wv) #[None, t, d]
        p = tf.matmul(q, k, transpose_b=True)/math.sqrt(self.d) # [None, t, t]
        
        if mask is None:
            p = tf.nn.softmax(p)
        else:
            p += mask * -1e9
            p = tf.nn.softmax(p)
                

        h = tf.matmul(p, v) # [None, t, t] . [None, t, d] => [None, t, d]
        return h,p

layer = SelfAttentionLayer(512)
mask = 1 - tf.linalg.band_part(tf.ones((7, 7)), -1, 0)
h, p = layer(x, x, x, mask)
print(h.shape)

(1, 7, 512)


## Defining a Mask using TensorFlow

In [26]:
# Controlling decimal formatting
#np.set_printoptions(precision=3)
#print(p.numpy())

mask = 1 - tf.linalg.band_part(tf.ones((7, 7)), -1, 0)
print(mask)

tf.Tensor(
[[0. 1. 1. 1. 1. 1. 1.]
 [0. 0. 1. 1. 1. 1. 1.]
 [0. 0. 0. 1. 1. 1. 1.]
 [0. 0. 0. 0. 1. 1. 1.]
 [0. 0. 0. 0. 0. 1. 1.]
 [0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0.]], shape=(7, 7), dtype=float32)


In [10]:
multi_attn_head = [SelfAttentionLayer(64) for i in range(8)]
outputs = [head(x, x, x)[0] for head in multi_attn_head]
outputs = tf.concat(outputs, axis=-1)
print(outputs.shape)

(1, 7, 512)


## Defining the Encoder Layer and the Decoder Layer

In [50]:
class EncoderLayer(layers.Layer):
    
    def __init__(self, d, n_heads):
        super(EncoderLayer, self).__init__()
        self.d = d
        self.d_head = int(d/n_heads) 
        self.n_heads = n_heads
        self.attn_heads = [SelfAttentionLayer(self.d_head) for i in range(self.n_heads)]
        self.fc_layer = FCLayer(2048, self.d)
 
    def call(self, x):
        print(x.shape)
        def compute_multihead_output(x):
            outputs = [head(x, x, x)[0] for head in self.attn_heads]            
            outputs = tf.concat(outputs, axis=-1)
            return outputs

        h1 = compute_multihead_output(x)
        y = self.fc_layer(h1)
        
        return y

class DecoderLayer(layers.Layer):
    
    def __init__(self, d, n_heads):
        super(DecoderLayer, self).__init__()
        self.d = d
        self.d_head = int(d/n_heads)
        self.dec_attn_heads = [SelfAttentionLayer(self.d_head) for i in range(n_heads)]
        self.attn_heads = [SelfAttentionLayer(self.d_head) for i in range(n_heads)]
        self.fc_layer = FCLayer(2048, self.d)
        
    def build(self, input_shape):
        [head.build(input_shape) for head in self.dec_attn_heads]
        [head.build(input_shape) for head in self.attn_heads]
        self.fc_layer.build(input_shape)
        
    def call(self, de_x, en_x, mask=None):
        
        def compute_multihead_output(de_x, en_x, mask=None):
            outputs = [head(en_x, en_x, de_x, mask)[0] for head in self.attn_heads]
            outputs = tf.concat(outputs, axis=-1)
            return outputs
        
        h1 = compute_multihead_output(de_x, de_x, mask)        
        h2 = compute_multihead_output(h1, en_x)
        y = self.fc_layer(h2)
        return y


## Defining a Simple Transformer Network

In [55]:
n_steps = 25
n_en_vocab = 300
n_de_vocab = 400
n_heads = 8
d = 512
mask = 1 - tf.linalg.band_part(tf.ones((n_steps, n_steps)), -1, 0)

en_inp = layers.Input(shape=(n_steps,))
en_emb = layers.Embedding(n_en_vocab, 512, input_length=n_steps)(en_inp)
print(en_emb)
en_out1 = EncoderLayer(d, n_heads)(en_emb)
en_out2 = EncoderLayer(d, n_heads)(en_out1)

de_inp = layers.Input(shape=(n_steps,))
de_emb = layers.Embedding(n_de_vocab, 512, input_length=n_steps)(de_inp)

de_out1 = DecoderLayer(d, n_heads)(de_emb, en_out2, mask)
de_out2 = DecoderLayer(d, n_heads)(de_out1, en_out2, mask)
de_pred = layers.Dense(n_de_vocab, activation='softmax')(de_out2)

transformer = models.Model(inputs=[en_inp, de_inp], outputs=de_pred, name='MinTransformer')
transformer.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])
transformer.summary()

Tensor("embedding_22/Identity:0", shape=(None, 25, 512), dtype=float32)
(None, 25, 512)
(None, 25, 512)
Model: "MinTransformer"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_23 (InputLayer)           [(None, 25)]         0                                            
__________________________________________________________________________________________________
embedding_22 (Embedding)        (None, 25, 512)      153600      input_23[0][0]                   
__________________________________________________________________________________________________
input_24 (InputLayer)           [(None, 25)]         0                                            
__________________________________________________________________________________________________
encoder_layer_22 (EncoderLayer) (None, 25, 512)      2099712     embedding_22[0]